The source for this data is the Chadwick Baseball Bureau, and the entire data bank can be found here: https://github.com/chadwickbureau/baseballdatabank.


In [ ]:
import os

assert os.path.exists('baseball.db')

In [ ]:
%load_ext sql
%sql sqlite:////content/baseball.db

Here is the schema for `baseball.db`:

appearances
   (
   yearID: int,
   teamID: varchar(8),
   playerID: varchar(20),
   games_played: int
   )

parks
   (
   parkID: varchar(8),
   name: varchar(200),
   city: varchar(200),
   state: varchar(20),
   country: varchar(20)
   )

players
   (
   playerID: varchar(20),
   nameFirst: varchar(200),
   nameLast: varchar(200),
   weight: int,
   height: int
   )

teams
   (
   yearID: int,
   teamID: varchar(20),
   wins: int,
   losses: int,
   name: varchar(200),
   park: varchar(200)
   )

If you have any questions about what these mean, we recommend that you try referencing the source of the data, linked above.

In [ ]:
%%sql
-- To Find all teams that have had their park in Illinois
SELECT DISTINCT t.name AS "Team Name", p.name AS "Park Name", p.city AS "City"
FROM teams t
JOIN parks p ON t.park = p.name
WHERE state = "IL"

 * sqlite:////content/baseball.db
Done.


Team Name,Park Name,City
Chicago White Stockings,Lake Front Park I,Chicago
Chicago White Stockings,Lake Front Park II,Chicago
Chicago Colts,South Side Park I,Chicago
Chicago White Sox,South Side Park II,Chicago
Chicago Chi-Feds,Wrigley Field,Chicago
Chicago Cubs,Wrigley Field,Chicago
Chicago Whales,Wrigley Field,Chicago
Chicago White Sox,Guaranteed Rate Field,Chicago
Rockford Forest Citys,Agricultural Society Fair Grounds,Rockford


In [ ]:
%%sql
-- To find the shortest MLB player
SELECT nameFirst AS "First Name", nameLast AS "Last Name"
FROM players
WHERE height IN (SELECT min(height) FROM players)

 * sqlite:////content/baseball.db
Done.


First Name,Last Name
Eddie,Gaedel


In [ ]:
%%sql
-- To Find the total wins from the Minnesota Twins
SELECT sum(wins) AS "Total Wins"
FROM teams
WHERE teamId = "MIN";


 * sqlite:////content/baseball.db
Done.


Total Wins
4789


In [ ]:
%%sql
-- To find the entire roster of the 1987 world series champion Minnesota Twins
SELECT nameFirst AS "First Name", nameLast AS "Last Name"
FROM appearances app
LEFT JOIN players p ON (app.playerId = p.playerId)
WHERE teamId = "MIN" AND yearID = 1987

 * sqlite:////content/baseball.db
Done.


First Name,Last Name
Allan,Anderson
Keith,Atherton
Don,Baylor
Billy,Beane
Juan,Berenguer
Jeff,Bittiger
Bert,Blyleven
Tom,Brunansky
Randy,Bush
Sal,Butera


In [ ]:
%%sql
-- To Find the total number of parks that have ever been in each state
SELECT state, count(state) AS "Number of Parks"
FROM parks
GROUP BY state

 * sqlite:////content/baseball.db
Done.


state,Number of Parks
,1
AZ,1
CA,10
CO,2
CT,5
DC,11
DE,2
ENG,1
FL,4
GA,3


In [ ]:
%%sql
-- To find franchises with more than 10,000 total wins.
SELECT name AS "Franchise", SUM(wins) AS "Amount of Wins"
FROM teams
GROUP BY teamID
HAVING SUM(wins) > 10000

In [ ]:
%%sql
-- To find players that have played for more than four or more different teams in the same year
SELECT nameFirst as "First Name", nameLast AS "Last Name"
FROM players p
JOIN (
  SELECT *
  FROM appearances
  GROUP BY playerID, yearID
  HAVING COUNT(DISTINCT teamID) >= 4
) AS "app"
ON p.playerID = app.playerID


In [ ]:
%%sql
-- To find the complete roster of every team in the first year of the MLB
SELECT DISTINCT t.name AS "Team", c.nameFirst AS "First Name", c.nameLast AS "Last Name"
FROM teams t
JOIN (
  SELECT teamId, nameFirst, nameLast
  FROM Appearances app
  INNER JOIN Players p ON app.playerID = p.playerID
  where yearID = (SELECT min(yearID) FROM Appearances)) as c
ON (t.teamID = c.teamID)


In [ ]:
%%sql
-- To find every player who has played 25 or more years in the MLB
SELECT DISTINCT nameFirst AS "First Name", nameLast AS "Last Name", "Total Years Played"
FROM players p
JOIN(
  SELECT DISTINCT playerID, count(DISTINCT yearID) AS "Total Years Played"
  FROM appearances
  GROUP BY playerID
  HAVING count(DISTINCT yearID) >= 25
) AS "total_years"
ON p.playerID = total_years.playerID


In [ ]:
%%sql
-- To find the total number of teamamtes that Kirby Puckett had
SELECT count(playerID) AS "Kirby Puckett's Total Teammates"
FROM (
  SELECT DISTINCT (appearances.playerid)  FROM appearances
  INNER JOIN(
    SELECT app.teamid, app.yearid FROM appearances app
    LEFT JOIN players p ON app.playerid = p.playerid
    WHERE p.nameFirst = "Kirby" AND p.nameLast = "Puckett"
  ) AS "b"
  ON b.teamId = appearances.teamId AND b.yearId = appearances.yearId


)
